In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt

directory = Path(r"C:\Users\Niebaum\Desktop\dataset")

In [ ]:
drop_sondes = xr.open_dataset(directory / "drop_sondes.nc")
distance = xr.open_dataset(directory / "distance_dropsondes_identified_clusters_rain_mask_5.nc")
safir = xr.open_dataset(directory / "EUREC4A_SAFIRE_ATR_Core_1Hz_20200202_as200009_L2_V2.nc")
cloud_composite = xr.open_dataset(directory / "train_cc.nc")


safir["time"] = safir["time"].astype("datetime64[s]")
cloud_composite["time"] = cloud_composite["time"].astype("datetime64[s]")

In [ ]:
time_slice = slice("2020-02-02 14:55", "2020-02-02 15:10")
new_time = cloud_composite["time"].sel(time=time_slice)
new_time_slice = slice(new_time.min(), new_time.max() + np.timedelta64(500, "ms"))

In [ ]:
ds1 = safir.sel(time=new_time_slice)
ds2 = cloud_composite.sel(time=new_time_slice)
len(ds1.time), len(ds2.time)

(78, 78)

In [ ]:
plt.plot(ds1["LWC"], 1e3 * ds2["liquid_water_content"], marker="x", linestyle="None")
# plot 1 to 1 line
plt.plot([0, 1], [0, 1])
plt.xlim([0, 0.7])
plt.ylim([0, 0.7])

(0.0, 0.7)

In [ ]:
ds1["ABS_HU1"]

In [ ]:
plt.scatter(
    ds1["ABS_HU1"],
    ds1["ABS_HU2"],
    marker="x",
    linestyle="None",
)

In [ ]:
def absolte_to_specific(
    absolute_humidity: xr.DataArray,
    pressure: xr.DataArray,
    temperature: xr.DataArray,
    R_v: float = 461,  # J K-1 kg-1
    epsilon: float = 0.622,
):

    e = R_v * absolute_humidity * temperature

    return epsilon * e / (pressure)  #   - e * (epsilon - 1))


q2 = absolte_to_specific(ds1["ABS_HU1"], ds1["PRESSURE"], ds1["TEMPERATURE"])

In [ ]:
# q1.plot()
q2.plot()